# Mathematisches Pendel

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Die Bewegungsgleichung des mathematischen Pendels lautet in dimensionsloser Form

$$\frac{\text{d}^2\theta}{\text{d}t^2} = -\sin(\theta)\,.$$

Obwohl sich diese Gleichung analytisch lösen lässt, soll sie hier mit Hilfe der SciPy-Funktion `integrate.solve_ivp` numerisch gelöst werden. Neben der zeitabhängigen Bewegung wird auch die Bewegung im Phasenraum betrachtet.

Da die Gesamtenergie des mathematischen Pendels erhalten ist, wird im zweiten Abschnitt die Qualität der numerischen Lösung anhand der zeitlichen Änderung der Gesamtenergie beurteilt.

## Lösung der Bewegungsgleichung und Phasenraumdarstellung

### Importanweisungen

In [ ]:
from math import cos, pi, sin
import numpy as np
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementation der Bewegungsgleichung

Wie schon in vorhergehenden Jupyter-Notebooks muss die Differentialgleichung 2. Ordnung mit Hilfe von Differentialgleichungen 1. Ordnung dargestellt werden. Diese lauten

\begin{align}
\frac{\text{d}\theta}{\text{d}t} &= v_\theta\\
\frac{\text{d}v_\theta}{\text{d}t} &= -\sin(\theta)
\end{align}

und werden in der folgenden Funktion umgesetzt.

In [ ]:
def dtheta_dt(t, theta_vec):
    theta, v_theta = theta_vec
    return v_theta, -sin(theta)

### Lösung des Differentialgleichungssystems

In der folgenden Zelle wird `integrate.solve_ivp` benutzt, um ähnlich wie in vorhergehenden Jupyter-Notebooks die Bewegungsgleichung zu lösen. Neu ist hier die Verwendung von `atol` und `rtol`, um Schranken für den absoluten bzw. den relativen Fehler festzulegen. Dabei genügt es, wenn eine der beiden Fehlerschranken unterschritten wird.

Um ein Phasenraumdiagramm erstellen zu können, werden neben den verwendeten Zeitpunkten alle Bestandteile des Lösungsvektors `solution.y` übergeben, die sowohl den zeitabhängigen Winkel $\theta(t)$ als auch die zeitabhängige Winkelgeschwindigkeit $v_\theta(t)$ umfassen, übergeben. Außerdem wird die Anzahl der Auswertungen von `dtheta_dt` übergeben.

In [ ]:
def trajectory(t_end, n_out, theta_0, v_theta_0,
               atol, rtol):
    t_values = np.linspace(0, t_end, n_out)
    solution = integrate.solve_ivp(
        dtheta_dt, (0, t_end), [theta_0, v_theta_0],
        t_eval=t_values, atol=atol, rtol=rtol)
    theta, v_theta = solution.y[:]
    t = solution.t
    return t, theta, v_theta, solution.nfev

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Es können die folgenden Parameter eingestellt werden:
- `t_end`: Länge des Zeitintervalls
- `n_out`: Anzahl der Datenpunkte
- `theta_0_over_pi`: Anfangswinkel dividiert durch π. Die Werte 0 und 1 entsprechen somit der stabilen bzw. der instabilen Gleichgewichtslage.
- `v_theta_0`: anfängliche Winkelgeschwindigkeit
- `atol`: absoluter Fehler
- `rtol`: relativer Fehler
- `plottype`: Art der Darstellung, entweder Zeitabhängigkeit $\theta(t)$ oder Phasenraumdarstellung $v_\theta(\theta)$

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=10, min=0.1, max=30, step=0.1,
                   description=r"$t_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=100, min=10, max=1000, step=10,
                   description=r"$n_\text{out}$"),
               "theta_0_over_pi":
               widgets.FloatSlider(
                   value=0.6, min=-1, max=1, step=0.05,
                   description=r"$\theta(0)/\pi$"),
               "v_theta_0":
               widgets.FloatSlider(
                   value=0, min=-2.5, max=2.5, step=0.1,
                   description=r"$v_\theta(0)$"),
               "atol":
               widgets.FloatLogSlider(
                   value=1e-6, min=-12, max=-2, step=0.1,
                   readout_format=".2e",
                   description="atol"),
               "rtol":
               widgets.FloatLogSlider(
                   value=1e-6, min=-12, max=-2, step=0.1,
                   readout_format=".2e",
                   description="rtol"),
               "plottype":
               widgets.RadioButtons(
                   options=["Zeitabhängigkeit",
                            "Phasenraum"],
                   description="Darstellung")
               }

@interact(**widget_dict)
def plot_dynamics(t_end, n_out, theta_0_over_pi, v_theta_0,
                  atol, rtol, plottype):
    theta_0 = pi * theta_0_over_pi
    t, theta, v_theta, count = trajectory(
        t_end, n_out, theta_0, v_theta_0, atol, rtol)

    fig, ax = plt.subplots()
    if plottype == "Zeitabhängigkeit":
        ax.plot(t, theta, label=r"$\theta(t)$")
        ax.plot(t, v_theta, label=r"$v_\theta(t)$")
        ax.set_xlabel("$t$")
        ax.legend(loc="lower right")
    else:
        ax.plot(theta, v_theta)
        ax.set_xlabel(r"$\theta$")
        ax.set_ylabel(r"$v_\theta$")
    print(f"{count} Aufrufe von dtheta_dt")

## Kontrolle anhand der Verletzung der Energieerhaltung als Funktion der Zeit

Während die Lösung der Bewegungsgleichung den Energieerhaltungssatz erfüllt, ist dies für die numerische Lösung im Allgemeinen nicht der Fall. Daher wird die zeitliche Änderung der Energie betrachtet, um die Qualität der numerischen Lösung beurteilen zu können. Interessant ist es dabei, den Einfluss der Änderung der Fehlerschranken `atol` und `rtol` zu betrachten.

Eine Veränderung der Kontrollparameter wirkt sich auch in der Darstellung der berechneten Lösung aus.

In [ ]:
@interact(**widget_dict)
def plot_energy(t_end, n_out, theta_0_over_pi, v_theta_0,
                atol, rtol):
    theta_0 = pi * theta_0_over_pi
    energy_0 = v_theta_0**2/2-cos(theta_0)
    t, theta, v_theta, count = trajectory(
        t_end, n_out, theta_0, v_theta_0, atol, rtol)
    delta_e = v_theta**2/2-np.cos(theta) - energy_0

    fig, ax = plt.subplots()
    ax.plot(t, delta_e)
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\Delta E$")